In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# PDFドキュメントの読込
documents = SimpleDirectoryReader('./data/pdf').load_data()


In [3]:
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Settings
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=256,
    chunk_overlap=16,
    tokenizer=tiktoken.encoding_for_model(MODEL_NAME).encode)

# 言語モデルの指定
llm = OpenAI(model=MODEL_NAME, temperature=0.3)

# 設定に反映
Settings.llm = llm
Settings.node_parser = node_parser

# Indexの構築
index = VectorStoreIndex.from_documents(documents)

In [4]:
# ストレージに保存
index.storage_context.persist("./storage01")

In [5]:
from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage01")
# Indexのロード
index = load_index_from_storage(storage_context)

In [6]:
sys_prompt_str = """
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

qa_prompt_str = """
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問: {query_str}
回答："""

refine_prompt_str = """
元の回答を (必要な場合のみ) 以下のコンテキストで改良する機会があります。
-----------
{context_msg}
-----------
新しいコンテキストが与えられた場合、元の回答を改良して、質問 {query_str} に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答: {existing_answer}"""

In [7]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# テキストQAテンプレートの作成
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# リファインテンプレートの作成
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [8]:
# Chat Engineの作成
chat_engine = index.as_chat_engine(
    chat_mode="openai",
    llm=llm,
    similarity_top_k=3,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)


In [9]:
# 質問：1回目
response = chat_engine.stream_chat("公共交通機関の交通費の上限は？")

for token in response.response_gen:
    print(token, end="")

公共交通機関の交通費の上限は、月額3万円です。

In [10]:
# 引用元を表示
for source in response.sources:
    for source_node in source.raw_output.source_nodes:
        print("ファイル名：", source_node.metadata["file_name"])
        print("関連度スコア:", source_node.score)
        print("テキスト：")
        print(source_node.node.text)
        print("-" * 50)  # 区切り線

ファイル名： 02賃金規則.pdf
関連度スコア: 0.8721262789345141
テキスト：
通勤⼿当
通勤にかかる交通費は、実際の経路に基づき⽀給されます。
公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。
⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使
⽤料やガソリン代の⼀部が⽀給される場合もあります。
2.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.8146777350506438
テキスト：
住宅⼿当
会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。
住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必
要です。
3. 家族⼿当
扶養家族がいる従業員には、家族⼿当が⽀給されます。
配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3
⼈まで）。
4.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.7872726375719755
テキスト：
休⽇に出勤する場合には、休⽇出勤⼿当が⽀給されます。
休⽇出勤⼿当の割増率は 1.35 倍です。
3. 賞与（ボーナス）
1. 賞与の⽀給基準
みらいテクノロジー株式会社では、年 2 回（ 6 ⽉と 12 ⽉）に賞与を⽀給します。
賞与の⾦額は、会社の業績や個⼈の評価結果に基づき決定されます。業績が好調な
場合は、賞与の額が増えることもあります。
2.
--------------------------------------------------


In [11]:
# 質問：2回目
response = chat_engine.stream_chat("交通費以外の手当にはどのようなものがありますか？")

for token in response.response_gen:
    print(token, end="")

交通費以外の手当には、以下のものがあります。

1. **住宅手当**: 通勤に1時間以上かかる場合に月額1万円が支給されます。居住地を証明できる書類の提出が必要です。

2. **家族手当**: 扶養家族がいる従業員に支給され、配偶者には月額5,000円、子供1人につき月額3,000円が支給されます（上限は子供3人まで）。

3. **時間外手当（残業代）**: 所定の勤務時間を超えて働いた場合に支給され、通常の時間外労働は1.25倍、深夜時間帯の残業は1.5倍となります。

4. **休日出勤手当**: 休日に出勤した場合に支給される手当です。

In [12]:
# 引用元を表示
for source in response.sources:
    for source_node in source.raw_output.source_nodes:
        print("ファイル名：", source_node.metadata["file_name"])
        print("関連度スコア:", source_node.score)
        print("テキスト：")
        print(source_node.node.text)
        print("-" * 50)  # 区切り線

ファイル名： 02賃金規則.pdf
関連度スコア: 0.8419876141577401
テキスト：
通勤⼿当
通勤にかかる交通費は、実際の経路に基づき⽀給されます。
公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。
⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使
⽤料やガソリン代の⼀部が⽀給される場合もあります。
2.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.8055602847655532
テキスト：
住宅⼿当
会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。
住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必
要です。
3. 家族⼿当
扶養家族がいる従業員には、家族⼿当が⽀給されます。
配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3
⼈まで）。
4.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.7984035894909728
テキスト：
時間外⼿当（残業代）
所定の勤務時間を超えて働いた場合は、残業⼿当が⽀給されます。
残業⼿当の割増率は、法令に基づき計算されます。通常の時間外労働は 1.25 倍、深
夜時間帯（午後 10 時以降）の残業は 1.5 倍となります。
5. 休⽇出勤⼿当
--------------------------------------------------
